# Base64 encoding MyPHD keys 

This script leverages PyCap to do the following:

1. Downloads MyPHD keys from a given project
2. Encodes them as base64
3. Uploads resulting string as a field for each record

### Setup



In [ ]:
# Create and activate virtual environment
! python3 -m venv venv
! source ./venv/bin/activate venv

# Install kernel 
! pip install ipykernel
! python3 -m ipykernel install --user 

# Create environmental file, follow the example to fill in the necessary variables
! touch .env

# If you are using VsCode: Ensure in python3 venv interpreter is selected with Ctrl+Shift+P

### Install dependencies
- Pycap is the package used to integrate with the REDCap API
- Dotenv is a package used to handle environmental variables
- If Pycap and Dotenv packages already installed on your system the script will attempt to update them.

In [1]:

! pip install --upgrade pycap
! pip install --upgrade python-dotenv

  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 8.8 MB/s eta 0:00:00
  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)


## Script Execution

### Import dependencies
  - Make sure to update `.env` if saved somewhere else.
  - Attribute `override` will prevent caching of `.env` values and reload them everytime script runs.

In [2]:
import os
import base64
import logging
from datetime import datetime
from dotenv import load_dotenv
from redcap import Project, RedcapError
logging.getLogger().setLevel(logging.INFO)
load_dotenv('.env', override=True)
logging.info('Main libraries are loaded')

INFO:root:Main libraries are loaded


### Default variable fields
These fields can be set according to your REDCap project

In [3]:
local = True # Whether or not to attempt to handle keys in a local folder 

recordFileField = 'upload' # Field to pull from, used if local is False
folderFilePath = '/Users/jmschult/Desktop/Work/Pycap/testfiles' # Absolute path to key folder, used if local is True

recordPrimaryKey = 'record_id' # General ID field of record
recordUploadField = 'key_base64' # Field to upload resulting data to

In [ ]:
error = []

if not os.getenv('REDCAP_API_KEY') or not os.getenv('REDCAP_API_URL'):
    error.append('Environmental variables have not been set. Please update your .env file')
else:
    try:
        # Initialize connection to project
        project = Project(str(os.getenv('REDCAP_API_URL')), str(os.getenv('REDCAP_API_KEY')))
        newRecords = []
        files = 0

        if not local: # Attempt to fetch key information from REDCap
            # Grab all record IDs
            records = project.export_records(
                format_type='json',
                fields=[recordPrimaryKey],
                raw_or_label='raw'
            )

            for record in records:
                if(int(record[recordPrimaryKey])):
                    # Grab key file from record
                    logging.getLogger('process').info('Processing Record Id:' + record[recordPrimaryKey])
                    key = project.export_file(
                        record=record[recordPrimaryKey],
                        field=recordFileField
                    )

                    if key:
                        # encode as base64, convert to string representation and store
                        encoded = base64.b64encode(key[0])
                        newRecord = {
                            recordPrimaryKey: int(record[recordPrimaryKey]),
                            recordUploadField: encoded.decode('ascii')
                        }
                        newRecords.append(newRecord)
                    else:
                        error.append(f'Record {record[recordPrimaryKey]} has no file uploaded with the field name {recordFileField}')
            files = len(records)
        
        else: 
            with os.scandir(folderFilePath) as it:
                for entry in it: 
                    files+=1
                    if entry.is_file():
                        with open(entry.path, 'r') as f:
                            data = f.read()
                            bytes = data.encode('ascii') # data must be encoded before base64
                            encoded = base64.b64encode(bytes)
                            newRecord = {
                                recordPrimaryKey: entry.name,
                                recordUploadField: encoded.decode('ascii')
                            }
                            newRecords.append(newRecord)
        # upload result                            
        result = project.import_records(newRecords)
        logging.getLogger('process').info(str(result['count']) + ' out of ' + str(files) + ' records are successfully processed!')
        
    except RedcapError as e:
        error.append(f'REDCap Error : {e}')
    except ValueError as e:
        error.append(f'Value Error : {e}')

# Log errors if any
if len(error):
    time = datetime.now().strftime('%B %d, %Y %H:%M:%S')
    logging.error(f'\n\n####### Error log for script execution on {time}\n')
    logging.error('\n'.join(error))